## Evaluator-Optimizer Workflow
이 워크플로우에서는 하나의 LLM이 응답을 생성하고 다른 LLM이 순환적으로 평가와 피드백을 제공합니다.

### 이 워크플로우의 적용 시점
이 워크플로우는 다음과 같은 상황에서 특히 효과적입니다:

- 명확한 평가 기준이 있는 경우
- 반복적인 개선이 가치를 창출하는 경우

적합성을 판단하는 두 가지 지표는 다음과 같습니다:

- 피드백이 제공될 때 LLM 응답이 입증 가능한 수준으로 개선되는 경우
- LLM이 스스로 의미 있는 피드백을 제공할 수 있는 경우

In [1]:
from util import llm_call, extract_xml

def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")
    
    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")
    
    return thoughts, result

def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")
    
    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")
    
    return evaluation, feedback

def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})
    
    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

### 예시 활용 사례: 반복적 코딩 루프


In [3]:
evaluator_prompt = """
다음 코드 구현을 아래 기준으로 평가하세요:
1. 코드 정확성
2. 시간 복잡도
3. 스타일 및 모범 사례

평가만 진행하고 과제 해결을 시도하지 마세요.
모든 기준이 충족되고 더 이상 개선 제안이 없는 경우에만 "PASS"를 출력하세요.
평가를 다음 형식으로 간단히 출력하세요.

<evaluation>PASS, NEEDS_IMPROVEMENT, 또는 FAIL</evaluation>
<feedback>
개선이 필요한 사항과 그 이유.
</feedback>
"""

generator_prompt = """
<user input>에 기반하여 과제를 완료하는 것이 목표입니다. 이전 생성에 대한
피드백이 있다면 이를 반영하여 해결책을 개선해야 합니다.

답변을 다음 형식으로 간단히 출력하세요:

<thoughts>
[과제와 피드백에 대한 이해, 개선 계획]
</thoughts>

<response>
[코드 구현 내용]
</response>
"""

task = """
<user input>
스택을 구현합니다:
1. push(x)
2. pop()
3. getMin()
모든 연산은 O(1)이어야 합니다.
</user input>
"""

loop(task, evaluator_prompt, generator_prompt)



=== GENERATION START ===
Thoughts:

상수 시간 O(1)으로 최소값을 얻으려면 별도의 최소값 스택이 필요합니다.
주 스택과 최소값 스택을 동기화하여 관리하면 모든 연산을 O(1)에 수행할 수 있습니다.


Generated:

```python
class MinStack:
    def __init__(self):
        self.stack = []        # 주 스택
        self.minStack = []     # 최소값 스택
        
    def push(self, x: int) -> None:
        self.stack.append(x)
        # 최소값 스택이 비어있거나 새 값이 현재 최소값보다 작거나 같으면 추가
        if not self.minStack or x <= self.minStack[-1]:
            self.minStack.append(x)
            
    def pop(self) -> None:
        if not self.stack:
            return
        # 제거되는 값이 현재 최소값이면 최소값 스택에서도 제거
        if self.stack[-1] == self.minStack[-1]:
            self.minStack.pop()
        self.stack.pop()
        
    def getMin(self) -> int:
        if not self.minStack:
            return None
        return self.minStack[-1]
```

=== GENERATION END ===

=== EVALUATION START ===
Status: PASS
Feedback: 
코드가 모든 요구사항을 충족하며 잘 구현되어 있습니다:

1. 정확성
- 모든 요구 기능(push, pop, getMin)이 올바르게 구현됨
-

('\n```python\nclass MinStack:\n    def __init__(self):\n        self.stack = []        # 주 스택\n        self.minStack = []     # 최소값 스택\n        \n    def push(self, x: int) -> None:\n        self.stack.append(x)\n        # 최소값 스택이 비어있거나 새 값이 현재 최소값보다 작거나 같으면 추가\n        if not self.minStack or x <= self.minStack[-1]:\n            self.minStack.append(x)\n            \n    def pop(self) -> None:\n        if not self.stack:\n            return\n        # 제거되는 값이 현재 최소값이면 최소값 스택에서도 제거\n        if self.stack[-1] == self.minStack[-1]:\n            self.minStack.pop()\n        self.stack.pop()\n        \n    def getMin(self) -> int:\n        if not self.minStack:\n            return None\n        return self.minStack[-1]\n```\n',
 [{'thoughts': '\n상수 시간 O(1)으로 최소값을 얻으려면 별도의 최소값 스택이 필요합니다.\n주 스택과 최소값 스택을 동기화하여 관리하면 모든 연산을 O(1)에 수행할 수 있습니다.\n',
   'result': '\n```python\nclass MinStack:\n    def __init__(self):\n        self.stack = []        # 주 스택\n        self.minStack = []     # 최소값 스택\n 